In [1]:
!which pip
!which python
# !pip install numpy torch scikit-learn tqdm
# !pip install pandas ultralytics matplotlib torchvision IPython opencv-python opencv-contrib-python

/workspace/venv/bin/pip
/workspace/venv/bin/python


In [2]:
import os
import numpy as np
import gc
import torch.distributed as dist
import pandas as pd
import torch.multiprocessing as mp
from torch.nn.parallel import DistributedDataParallel as DDP
import matplotlib.pyplot as plt
from typing import List
import cv2
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import accuracy_score, f1_score
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision
from torchvision import transforms
import pandas as pd
import json
from PIL.Image import Image
import PIL
from tqdm import tqdm
from PIL import ImageDraw
from ultralytics import YOLO

In [3]:
os.chdir("/workspace")

## Infer mordern

In [5]:
model = YOLO('mordern-weights.pt')
model("./test/mordern/images",save=True, conf=0.2, iou=0.5, save_txt=True)


image 1/200 /workspace/test/mordern/images/test_1016805_0080.jpg: 480x640 1 1_overall, 37 8_textlines, 84.2ms
image 2/200 /workspace/test/mordern/images/test_1019174_0012.jpg: 480x640 1 1_overall, 1 4_illustration, 1 6_headline, 26 8_textlines, 17.0ms
image 3/200 /workspace/test/mordern/images/test_1020077_0078.jpg: 480x640 1 1_overall, 2 6_headlines, 34 8_textlines, 17.0ms
image 4/200 /workspace/test/mordern/images/test_1023230_0031.jpg: 480x640 1 1_overall, 2 6_headlines, 44 8_textlines, 17.0ms
image 5/200 /workspace/test/mordern/images/test_1025409_0065.jpg: 480x640 1 1_overall, 1 4_illustration, 8 6_headlines, 56 8_textlines, 16.9ms
image 6/200 /workspace/test/mordern/images/test_1027320_0011.jpg: 480x640 1 1_overall, 2 4_illustrations, 1 6_headline, 2 7_captions, 34 8_textlines, 16.9ms
image 7/200 /workspace/test/mordern/images/test_1027421_0076.jpg: 480x640 1 1_overall, 27 8_textlines, 17.0ms
image 8/200 /workspace/test/mordern/images/test_1032573_0004.jpg: 480x640 1 1_overall, 

[ultralytics.yolo.engine.results.Results object with attributes:
 
 _keys: ('boxes', 'masks', 'probs')
 boxes: ultralytics.yolo.engine.results.Boxes object
 keys: ['boxes']
 masks: None
 names: {0: '0_background', 1: '1_overall', 2: '2_handwritten', 3: '3_typography', 4: '4_illustration', 5: '5_stamp', 6: '6_headline', 7: '7_caption', 8: '8_textline', 9: '9_table'}
 orig_img: array([[[255, 255, 255],
         [255, 255, 255],
         [255, 255, 255],
         ...,
         [255, 255, 255],
         [255, 255, 255],
         [255, 255, 255]],
 
        [[255, 255, 255],
         [255, 255, 255],
         [255, 255, 255],
         ...,
         [255, 255, 255],
         [255, 255, 255],
         [255, 255, 255]],
 
        [[255, 255, 255],
         [255, 255, 255],
         [255, 255, 255],
         ...,
         [255, 255, 255],
         [255, 255, 255],
         [255, 255, 255]],
 
        ...,
 
        [[255, 255, 255],
         [255, 255, 255],
         [255, 255, 255],
         .

## Infer old

In [6]:
model = YOLO('old-weights.pt')
model("./test/old/images",save=True, conf=0.2, iou=0.5, save_txt=True)


image 1/211 /workspace/test/old/images/test_10303536_0001.jpg: 480x640 1 1_overall, 1 2_handwritten, 2 3_typographys, 1 4_illustration, 2 5_stamps, 18.3ms
image 2/211 /workspace/test/old/images/test_10303703_0031.jpg: 480x640 1 1_overall, 21 3_typographys, 17.0ms
image 3/211 /workspace/test/old/images/test_10303741_0022.jpg: 480x640 1 1_overall, 8 2_handwrittens, 1 4_illustration, 1 5_stamp, 17.0ms
image 4/211 /workspace/test/old/images/test_10303784_0035.jpg: 480x640 1 1_overall, 23 2_handwrittens, 1 3_typography, 1 5_stamp, 17.0ms
image 5/211 /workspace/test/old/images/test_1100587_0003.jpg: 480x640 1 1_overall, 1 3_typography, 1 4_illustration, 17.0ms
image 6/211 /workspace/test/old/images/test_11038708_0010.jpg: 480x640 1 1_overall, 26 3_typographys, 3 4_illustrations, 4 5_stamps, 17.0ms
image 7/211 /workspace/test/old/images/test_11038723_0008.jpg: 480x640 1 1_overall, 1 2_handwritten, 46 3_typographys, 1 5_stamp, 17.0ms
image 8/211 /workspace/test/old/images/test_1122227_0003.jp

[ultralytics.yolo.engine.results.Results object with attributes:
 
 _keys: ('boxes', 'masks', 'probs')
 boxes: ultralytics.yolo.engine.results.Boxes object
 keys: ['boxes']
 masks: None
 names: {0: '0_background', 1: '1_overall', 2: '2_handwritten', 3: '3_typography', 4: '4_illustration', 5: '5_stamp', 6: '6_headline', 7: '7_caption', 8: '8_textline', 9: '9_table'}
 orig_img: array([[[183, 191, 190],
         [185, 193, 192],
         [188, 196, 195],
         ...,
         [195, 200, 199],
         [197, 202, 201],
         [199, 204, 203]],
 
        [[185, 193, 192],
         [186, 194, 193],
         [188, 196, 195],
         ...,
         [193, 198, 197],
         [194, 199, 198],
         [195, 200, 199]],
 
        [[189, 195, 194],
         [189, 195, 194],
         [189, 195, 194],
         ...,
         [195, 200, 199],
         [196, 201, 200],
         [197, 202, 201]],
 
        ...,
 
        [[184, 186, 187],
         [185, 187, 188],
         [182, 186, 187],
         .